In [40]:
# Find class labels
import os
import numpy as np
import re
import h5py
from PIL import Image, ImageOps
from fuel.converters.base import fill_hdf5_file
from scipy.misc import imread

In [39]:
Image

<module 'Image' from '/home/cipta/anaconda3/envs/theano2/lib/python2.7/site-packages/PIL/Image.pyc'>

In [2]:
!mkdir food-101-32x32
!mkdir food-101-64x64

In [13]:
labels = os.listdir('food-101/images')
label_ids = range(len(labels))

with open('food-101-labels.txt', 'w') as f:
    for l in labels:
        f.write(l)
        f.write('\n')

!head food-101-labels.txt

tuna_tartare
frozen_yogurt
spaghetti_carbonara
beignets
cannoli
chicken_quesadilla
ramen
churros
garlic_bread
guacamole


In [8]:
len(label_ids)

101

In [46]:
import os, sys
import re
import Image, ImageOps

size = 64
limit = 10

def read_in(size=64, limit=10, class_limit=10):
    vectors = []
    targets = []
    for i, c in zip(label_ids, labels)[:class_limit]:
        print(c)
        if c != 'steak':
            continue
        for j, fname in enumerate(os.listdir('food-101/images/' + c)[:limit]):
            print(fname)
            infile = os.path.join('food-101/images', c, fname)
            outfname = re.sub('jpe?g$', 'png', fname)
            outfile = 'food-101-{0}x{0}/{1}_{2}'.format(size, c, outfname)
            try:
                im = Image.open(infile)
                im = ImageOps.fit(
                    im,
                    (size, size),
                    Image.ANTIALIAS
                )
                if j < 10:
                    # Save the sample 10 images
                    im.save(outfile, "png")
                targets.append(i)
                imarr = np.array(im, dtype=np.uint8).transpose([2, 0, 1])
                vectors.append(imarr)
            except IOError:
                print "cannot create thumbnail for '%s'" % infile
                    
    vectors = np.stack(vectors, 0)
    targets = np.array(targets).reshape(-1, 1)
    
    data = (('train', 'features', vectors),
            ('train', 'targets', targets))
    
    h5file = h5py.File('food-101-{0}x{0}.hdf5'.format(size), mode='w')
    try:
        fill_hdf5_file(h5file, data)

        h5file['features'].dims[0].label = 'batch'
        h5file['features'].dims[1].label = 'channel'
        h5file['features'].dims[2].label = 'height'
        h5file['features'].dims[3].label = 'width'
        h5file['targets'].dims[0].label = 'batch'
        h5file['targets'].dims[1].label = 'index'

        h5file.flush()
    finally:
        h5file.close()
        
    return vectors, targets

In [48]:
v, l = read_in(size=64, limit=10000, class_limit=10000)

tuna_tartare
frozen_yogurt
spaghetti_carbonara
beignets
cannoli
chicken_quesadilla
ramen
churros
garlic_bread
guacamole
croque_madame
lobster_bisque
creme_brulee
caprese_salad
tacos
takoyaki
chocolate_cake
omelette
prime_rib
hot_dog
breakfast_burrito
fried_calamari
lasagna
shrimp_and_grits
waffles
miso_soup
seaweed_salad
red_velvet_cake
greek_salad
poutine
falafel
pho
grilled_salmon
sushi
bread_pudding
cheese_plate
pork_chop
peking_duck
bruschetta
strawberry_shortcake
deviled_eggs
clam_chowder
beef_tartare
panna_cotta
lobster_roll_sandwich
risotto
gnocchi
grilled_cheese_sandwich
spaghetti_bolognese
steak
465494.jpg
290406.jpg
234626.jpg
2324994.jpg
1650002.jpg
101312.jpg
2944161.jpg
2038418.jpg
1049459.jpg
1658443.jpg
221604.jpg
2765887.jpg
2825100.jpg
3002350.jpg
217250.jpg
1264050.jpg
3867460.jpg
830007.jpg
391144.jpg
2943601.jpg
1512226.jpg
1697339.jpg
3663800.jpg
1675632.jpg
1849463.jpg
184110.jpg
1248337.jpg
2979061.jpg
219196.jpg
296375.jpg
1289900.jpg
368073.jpg
2573392.jpg
2287

In [38]:
v.shape

(100, 3, 64, 64)

In [41]:
v, l = read_in(size=64, limit=2000, class_limit=2000)

tuna_tartare
frozen_yogurt
spaghetti_carbonara
beignets
cannoli
chicken_quesadilla


KeyboardInterrupt: 